In [1]:
import os

In [2]:
%pwd

'd:\\MLOPS\\End-To-End-MlOps-Project\\experiment'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\MLOPS\\End-To-End-MlOps-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataIngestionConfig:
    dir: Path
    url: Path
    local_dir: Path
    unzip_dir: Path

In [ ]:
from src.dsproject.utils.utils import read_yaml, create_dir

In [ ]:
from src.dsproject.constants.yaml_path import Config_file_path,Schema_file_path,Param_file_path

In [ ]:
from src.dsproject import logging
import requests

In [16]:
class ConfigManager:
    def __init__(self,config_file_path=Config_file_path,prams_file_path=Param_file_path,schema_file_path=Schema_file_path) -> None:
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(prams_file_path)
        self.schema=read_yaml(schema_file_path)

        create_dir([self.config.artifacts_root])
    
    def get_data_ingestion_config(self):
        try:
            config = self.config.Data_Ingestion
            data_ingestion_config = DataIngestionConfig(
                dir=config.dir,
                url=config.url,
                local_dir=config.local_dir,
                unzip_dir=config.unzip_dir
            )
            return data_ingestion_config
        except Exception as e:
            logging.info('error in data ingestion config', str(e))
            raise e

In [17]:
import zipfile
import urllib.request as rq

In [ ]:
import os
import requests
import zipfile
from src.dsproject.entity.config_entity import DataIngestionConfig
from src.dsproject import logging

class DataIngestion:
    def __init__(self, config=DataIngestionConfig) -> None:
        """
        Initializes the DataIngestion class with configuration settings.
        """
        self.config = config
        logging.info('************************* Data Ingestion **************************')

    def download_data(self):
        """
        Downloads data from the specified URL in the config and saves it locally.
        """
        try:
            data_url = self.config.url
            download_file_path = os.path.join(self.config.local_dir, 'data.zip')
            
            # Create a local data folder if it doesn't exist
            os.makedirs(self.config.local_dir, exist_ok=True)
            
            # Download the data
            response = requests.get(data_url)
            if response.status_code == 200:
                logging.info('Data download completed')
            else:
                logging.error('Failed to download data: Status code {}'.format(response.status_code))
                return

            # Save data to file
            with open(download_file_path, 'wb') as f:
                f.write(response.content)
            logging.info("Data saved successfully at {}".format(download_file_path))
        
        except Exception as e:
            logging.error('Error in data ingestion during download: {}'.format(str(e)))
            raise e

    def extract_data(self):
        """
        Extracts the downloaded ZIP data file into the specified directory.
        """
        try:
            download_file_path = os.path.join(self.config.local_dir, 'data.zip')
            unzip_dir = self.config.unzip_dir

            # Create an unzip directory if it doesn't exist
            os.makedirs(unzip_dir, exist_ok=True)

            # Unzip the downloaded file
            with zipfile.ZipFile(download_file_path, 'r') as zip_ref:
                zip_ref.extractall(unzip_dir)
            logging.info('Data unzip completed in {}'.format(unzip_dir))

        except zipfile.BadZipFile:
            logging.error('Failed to unzip file: BadZipFile error')
            raise
        except Exception as e:
            logging.error('Error in data ingestion during extraction: {}'.format(str(e)))
            raise e


In [19]:
try:
		config=ConfigManager()
		data_ingestion_config=config.get_data_ingestion_config()
		data_ingestion=DataIngestion(config=data_ingestion_config)
		data_ingestion.download_data()
		data_ingestion.extract_data()
except Exception as e:
         logging.info('error in data ingestion',str(e))
         raise e

[2024-11-05 13:45:03,854: INFO :utils : Yaml file config\config.yaml created]
[2024-11-05 13:45:03,858: INFO :utils : Yaml file params.yaml created]
[2024-11-05 13:45:03,862: INFO :utils : Yaml file schema.yaml created]
[2024-11-05 13:45:03,866: INFO :utils : created directory at: artifacts]
[2024-11-05 13:45:03,867: INFO :695794688 : ************************* Data Ingestion **************************]
[2024-11-05 13:45:04,526: INFO :695794688 : Data download completed]
[2024-11-05 13:45:04,537: INFO :695794688 : Data saved successfully at artifacts/data_ingestion/data.zip\data.zip]
[2024-11-05 13:45:04,572: INFO :695794688 : Data unzip completed in artifacts/data_ingestion/data]
